In [ ]:
#clears solution dir
import os, shutil
folder = './solutions'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
#clears solution_organized dir
import os, shutil
folder = './solutions_organized'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
#checks not completed problems
nc_file="not_completed/not_completed_problems.txt"
with open(nc_file, "r") as f:
    lines = f.readlines()
    f.close()

nc_problem_names = []
for l in lines:
    nc_problem_names.append(l.strip())

nc_problem_names = list(set(nc_problem_names))
print(len(nc_problem_names))

import os
solutions_path = "solutions"
problems_out_of_time = []
problems_with_no_solution = []
for problem in nc_problem_names:
    for file in os.listdir(solutions_path):
        if file.startswith(problem+"_output.sas"):
            with open(os.path.join(solutions_path, file), "r") as f:
                lines = f.readlines()
                f.close()
            time_taken = lines[-1:][0].strip()
            #regex to extract the time
            import re
            time_taken = re.search(r'Time taken: (\d+\.\d+)', time_taken)
            

            if time_taken:
                time_taken = float(time_taken.group(1))
                if time_taken > 3599:
                    problems_out_of_time.append(problem)
                    print(f"Problem {problem} took too long: {time_taken} seconds")
                else:
                    problems_with_no_solution.append(problem)
                    print(f"Problem {problem} has no solution, timed {time_taken} seconds")
            else:
                print("Time taken not found in the output.")
            break
print(f"Problems out of time: {len(problems_out_of_time)}")
print(f"Problems with no solution {len(problems_with_no_solution)}")
    

: 

In [ ]:
#counts completed FULL PROBLEMS (HAVE PXXX_0 TO PXXX_4 .SOL)
import os

solutions_path = "solutions"
completed_dict = {}
total_count_plans = 0
for file in os.listdir(solutions_path):
    if file.endswith(".SOL"):
        name = file.split("_")[0]
        if not (name in completed_dict):
            completed_dict[name] = 0
        completed_dict[name] += 1
    total_count_plans += 1

completed_count = 0
not_completed_count = 0
for key, val in completed_dict.items():
    if val >= 5:
        print(f"Problem: {key}, is completed ({val}/5)")
        completed_count += 1
    else:
        print(f"Problem: {key}, is NOT completed ({val}/5)")
        not_completed_count += 1

print("FULL PROBLEMS: ", completed_count)
print("NOT FULL PROBLEMS: ", not_completed_count)
print("TOTAL PLANS PROCESSED: ", total_count_plans)

In [ ]:
solutions_folder = './solutions'
og_solutions_folder = './og_solutions'

for sol_file in os.listdir(og_solutions_folder)
    target_path = os.path.join(solutions_folder, f"{sol_file.split(".")[0]}_og.SOL")
    shutil.copy(sol_file, target_path)

In [ ]:
# Organizes solutions into a structured directory based on problem completeness
# It copies .SOL and .sas files for problems that have at least 5 solutions into a new directory structure. Then goes through the original solutions and copies them into the corresponding directories.

import os
import re
import shutil

solutions_folder = './solutions'
target_sol_folder = './solutions_organized'

# Identify full problems based on having at least 6 .SOL files
completed_dict = {}
for file in os.listdir(solutions_folder):
    if file.endswith(".SOL"):
        problem = file.split(".")[0]
        if "_" in problem:
            problem = file.split("_")[0]
        completed_dict[problem] = completed_dict.get(problem, 0) + 1

full_problems = {problem for problem, cnt in completed_dict.items() if cnt == 6}
print("Full problems:", len(full_problems))

# Regex to match files like p123456_0.SOL or p123456_1.sas (case-insensitive)
pattern = re.compile(r'^(?P<problem>p\d+)_\d+.*\.(?P<ext>SOL|sas)$', re.IGNORECASE)

# Loop through all files in the solutions folder and copy only those for full problems
for filename in os.listdir(solutions_folder):
    file_path = os.path.join(solutions_folder, filename)
    if os.path.isfile(file_path):
        match = pattern.match(filename)
        if match:
            problem = match.group("problem")
            if problem in full_problems:
                ext = match.group("ext").lower()  # normalize extension to lower-case
                # Set target subdirectory based on file extension
                if ext == "sol":
                    subfolder = "sols"
                elif ext == "sas":
                    subfolder = "sas"
                else:
                    continue

                # Construct directory paths
                problem_folder = os.path.join(target_sol_folder, problem)
                target_folder = os.path.join(problem_folder, subfolder)
                os.makedirs(target_folder, exist_ok=True)

                # Build target file path and copy file
                target_file = os.path.join(target_folder, filename)
                # print(f"Copying {filename} to {target_file}")
                shutil.copy(file_path, target_file)

# Now, copy the original solutions from the og_solutions folder into the corresponding sols folder.
for problem_name in os.listdir(target_sol_folder):
    file_path = os.path.join(solutions_folder, f"{problem_name}.SOL")
    # Assuming original solution files have the .SOL extension
    if os.path.isfile(file_path) and file_path.upper().endswith(".SOL"):
        # Determine the problem prefix (assumes the naming format is similar, e.g. p123456_xxx)
        problem = file_path.split(".")[0]
        # print(f"Processing original solution file: {file_path}")
        if problem in full_problems:
            target_folder = os.path.join(target_sol_folder, problem, "sols")
            # print(target_folder)
            os.makedirs(target_folder, exist_ok=True)
            target_file = os.path.join(target_folder, f"{problem}_og.SOL")
            # print(f"Copying original {filename} from og_solutions to {target_file}")
            shutil.copy(file_path, target_file)

In [ ]:
#clears empty .sas coming from wrong pddls

import os
folder = './solutions'

files_to_remove = []
file_good = []
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    if file_path.endswith(".sas"):
        with open(file_path, 'r') as file:
            lines = file.readlines()
            file.close()
        if lines[0].startswith("Time"):
            files_to_remove.append(file_path)
        else:
            file_good.append(file_path)
    
for file in files_to_remove:
    os.remove(file)

In [ ]:
#checks that nc_problems actually have their corresponding file in solutions

nc_file="not_completed/not_completed_problems.txt"
with open(nc_file, "r") as f:
    lines = f.readlines()
    f.close()

nc_problem_names = []
for l in lines:
    nc_problem_names.append(l.strip())

nc_problem_names = list(set(nc_problem_names))
print(len(nc_problem_names))

import os
solutions_path = "./solutions"
problems_actually_nc = []
to_remove=[]
# print(nc_problem_names)
for problem in nc_problem_names:
    found = False
    # print(problem)
    for file in os.listdir(solutions_path):
        # print(file)
        if file.startswith(problem):
            problems_actually_nc.append(problem)
            found = True
            break
    if not found:
        to_remove.append(problem)
            
print(f"Problems nc: {len(problems_actually_nc)}")
print(f"Problems to remove {len(to_remove)}")

# string_to_write =""

# for p in problems_actually_nc:
#     string_to_write+=f"{p}\n"

# with open(nc_file, "w") as f:
#     f.write(string_to_write.strip())

In [ ]:
# New cell: Clear older log files leaving only the newest .txt and .err.txt per problem

import os, re
from collections import defaultdict

logs_folder = "logs"

# Updated regex: the entire string inside { } becomes the problem name (e.g. "p34142_3")
pattern = re.compile(
    r'^logs_script_\d+_plan\{(?P<problem>[^}]+)\.pddl\}(?P<err>\.err)?\.txt$'
)

# Build a dict mapping each problem name to the list of associated log files.
problem_files = defaultdict(list)

for fname in os.listdir(logs_folder):
    m = pattern.match(fname)
    if m:
        problem = m.group("problem")
        problem_files[problem].append(fname)

# For each problem, determine the newest plain log file and keep its pair (.err.txt if exists)
files_to_keep = set()

for problem, files in problem_files.items():
    # Filter out plain log files (exclude those ending with ".err.txt")
    plain_logs = [f for f in files if not f.endswith(".err.txt")]
    if not plain_logs:
        # If no plain log exists, use the available files
        plain_logs = files
    # Pick the plain log file with the most recent modification time
    ref_file = plain_logs[0]
    newest_time = os.path.getmtime(os.path.join(logs_folder, ref_file))
    for f in plain_logs[1:]:
        mtime = os.path.getmtime(os.path.join(logs_folder, f))
        if mtime > newest_time:
            newest_time = mtime
            ref_file = f
    files_to_keep.add(ref_file)
    # If an error log exists for this run, keep it too.
    err_candidate = ref_file.replace(".txt", ".err.txt")
    if err_candidate in files:
        files_to_keep.add(err_candidate)
    # If the reference file is an error log, also consider its plain log.
    if ref_file.endswith(".err.txt"):
        plain_candidate = ref_file.replace(".err.txt", ".txt")
        if plain_candidate in files:
            files_to_keep.add(plain_candidate)

files_to_keep = list(files_to_keep)
print(f"Keeping {len(files_to_keep)} log files out of {len(os.listdir(logs_folder))} total files.")

# Remove files not in the newest set for each problem.
for fname in os.listdir(logs_folder):
    if pattern.match(fname) and fname not in files_to_keep:
        os.remove(os.path.join(logs_folder, fname))
        print(f"Removed log file: {fname}")

In [ ]:
#take organized solution, create a class to represent a plan with initial state, actions and goals
#we have to start by iteratining on the organized solutions we have, and search the corrisponding problem pddl file in the dataset folder to extract initial state and goals
#actions can be taken from the .SOL file

import os
import re

class Plan:
    def __init__(self, problem_id, problem_full_name, initial_state, actions, goals):
        self.problem_id = problem_id
        self.problem_full_name = problem_full_name
        self.initial_state = initial_state
        self.actions = actions
        self.goals = goals

    def __repr__(self):
        return (f"Plan(problem_id={self.problem_id},\n"
                f"     problem_full_name={self.problem_full_name},\n"
                f"     initial_state={self.initial_state},\n"
                f"     actions={self.actions},\n"
                f"     goals={self.goals})")

def parse_pddl(pddl_file_path):
    """
    Parses a PDDL file to extract initial state and goals.
    This simple parser iterates through the file line by line.
    """
    initial_state = []
    goals = []
    mode = None
    try:
        with open(pddl_file_path, 'r') as f:
            for line in f:
                line_stripped = line.strip()
                # Start of initial state block
                if line_stripped.startswith("(:init"):
                    mode = "init"
                    # If there is content on the same line
                    remainder = line_stripped[len("(:init"):].strip()
                    if remainder and remainder != "(" and not remainder.startswith(";"):
                        initial_state.append(remainder)
                    continue
                # Start of goal block
                elif line_stripped.startswith("(:goal"):
                    mode = "goal"
                    remainder = line_stripped[len("(:goal"):].strip()
                    # Handle goal starting with (and ... if needed
                    if remainder.startswith("(and"):
                        remainder = remainder[len("(and"):].strip()
                        if remainder and remainder != "(" and not remainder.startswith(";"):
                            goals.append(remainder)
                    continue
                # End of a block
                elif line_stripped == ")" or line_stripped == ")))":
                    mode = None
                    continue

                # Append lines based on current block
                if mode == "init":
                    if line_stripped and not line_stripped.startswith(";"):
                        initial_state.append(line_stripped)
                elif mode == "goal":
                    if line_stripped and not line_stripped.startswith(";"):
                        goals.append(line_stripped)
    except Exception as e:
        print(f"Error parsing PDDL file {pddl_file_path}: {e}")
        raise Exception
    return initial_state, goals

def parse_sol(sol_file_path):
    """
    Parses a SOL file to extract the list of actions.
    Skips commented lines.
    """
    actions = []
    try:
        with open(sol_file_path, 'r') as f:
            for line in f:
                line_stripped = line.strip()
                # Skip comments (lines starting with ;)
                if line_stripped.startswith(";") or not line_stripped:
                    continue
                line_stripped = line_stripped.replace("(","").replace(")","")
                actions.append(line_stripped)
    except Exception as e:
        print(f"Error parsing SOL file {sol_file_path}: {e}")
    return actions

def main():
    # Base directories for organized solutions and dataset PDDL files.
    # Assumes the script is run from /home/rsignoroni/fastdownward_docker/
    solutions_organized_dir = "./solutions_organized"
    dataset_pddl_base = "./logistics/"  # adjust as needed

    plans = []

    # Iterate over each problem directory inside solutions_organized
    for problem in os.listdir(solutions_organized_dir):
        problem_dir = os.path.join(solutions_organized_dir, problem)
        if not os.path.isdir(problem_dir):
            continue

        sols_dir = os.path.join(problem_dir, "sols")
        if not os.path.exists(sols_dir):
            print(f"No 'sols' directory for problem {problem} in organized solutions.")
            continue

        # For simplicity, pick the first SOL file we find
        sol_files = [f for f in os.listdir(sols_dir) if f.upper().endswith(".SOL")]
        if not sol_files:
            print(f"No SOL files found in {sols_dir}")
            continue

        sol_file_path = os.path.join(sols_dir, sol_files[0])
        actions = parse_sol(sol_file_path)

        # Determine the corresponding PDDL file.
        # Eg: logistics/0-0.2/p000146/p000146_0.pddl
        pddl_file_path = None
        for folder in ["0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"]:
            candidate_path = os.path.join(dataset_pddl_base, folder, problem, f"{problem}_0.pddl")
            if os.path.exists(candidate_path):
                pddl_file_path = candidate_path
            break
        if pddl_file_path is None:
            print(f"PDDL file not found for problem {problem} in any subfolder.")
            continue

        initial_state, goals = parse_pddl(pddl_file_path)

        # Create a Plan object and add to the list.
        plan = Plan(problem_id=problem.split("_")[0], problem_version=problem initial_state=initial_state, actions=actions, goals=goals)
        plans.append(plan)

    # Print out all collected plans
    for plan in plans:
        print(plan)
        print("=" * 40)

if __name__ == "__main__":
    main()

In [ ]:
import os
sol_folder = './solutions'

for file in os.listdir(sol_folder):
    if "_" not in file:
        name = file.split(".")[0]
        new_name = f"{name}_og.SOL"
        os.rename(os.path.join(sol_folder,file), os.path.join(sol_folder,new_name))